In [2]:

from sklearn.model_selection import train_test_split
#from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import pandas as pd
import numpy as np
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Activation, Dropout, Flatten, Dense
import tensorflow as tf
from os import listdir
from PIL import Image
import glob, os
from os import listdir
from os.path import isfile, join
import numpy as np
from pandas import DataFrame
import pandas as pd
from matplotlib import pyplot as plt

In [3]:
# las funciones anteriores. Poner a parte e importar

In [3]:
def resize_image(imagen,size,output=False):
    im = Image.open(imagen)
    rgb_image = RGBA_a_RGB(im)
    im_rgb = rgb_image.resize(size)
    
    if not output:
        return im_rgb 
    onlyfiles = [f for f in listdir(output) if isfile(join(output, f))]
    if image not in onlyfiles:
        im.save(output)
        return im_rgb
    else:
        print("file already exists, continuing to next one") 
        return im_rgb

In [4]:
def RGBA_a_RGB(image1):
    if image1.mode == "RGBA":
        background = Image.new("RGB", image1.size, (255, 255, 255))
        background.paste(image1, mask = image1.split()[3])
        background.save(f"{image1}", "JPEG", quality=100)
        rgb_image = Image.open(f"{image1}")
        return rgb_image
    return image1

In [5]:
def image_to_array(image):
    im_np= np.asarray(image)
    one_line=im_np.ravel()
    return one_line

In [6]:
paths={"black":"INPUT/black/",
      "white":"INPUT/white/",
      "green":"INPUT/green/",
      "red":"INPUT/red/"}
images={}
for color,path in paths.items():
    images[color]=[]
    for im in listdir(path):
        if im.endswith("jpg"):
            #size=(64,64)
            a=resize_image(path+im,(64,64))
            images[color].append(np.asarray(a))
    images[color]=np.array(images[color])

In [7]:
def image_to_array(image):
    im_np= np.asarray(image)
    one_line=im_np.ravel()
    return one_line

In [8]:
X= np.concatenate(list(images.values()),axis=0)
y = np.concatenate([[k]* v.shape[0] for k,v in images.items()])

In [ ]:
# Aqui empieza realmente. Division entre la x y la y para test

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [11]:
img_width, img_height = 64, 64
img_rows, img_cols = 64, 64

# Prepare data to feed the NN
num_classes = 4

# Ask keras which format to use depending on used backend and arrange data as expected
if K.image_data_format() == 'channels_first':
    X_train = x_train.reshape(X_train.shape[0], 3, img_rows, img_cols)
    X_test = x_test.reshape(X_test.shape[0], 3, img_rows, img_cols)
    input_shape = (3, img_width, img_height)
else:
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 3)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 3)
    input_shape = (img_width, img_height, 3)

# Incoming data is in uint8. Cast the input data images to be floats in range [0.0-1.0]  
X_train = tf.keras.utils.normalize(X_train,axis=1)
X_test = tf.keras.utils.normalize(X_test,axis=1)
#X_train = X_train.astype('float32') / 255
#X_test = X_test.astype('float32') / 255

print('x_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

x_train shape: (8388, 64, 64, 3)
8388 train samples
2097 test samples


In [12]:
img_width, img_height = 64, 64

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(tf.keras.layers.Flatten())

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(4))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [16]:

 from sklearn.preprocessing import OneHotEncoder

In [13]:
##1
y_test


array(['white', 'white', 'black', ..., 'red', 'white', 'red'], dtype='<U5')

In [14]:
###2 Hacemos el reshape, luego onehot encoder, luego fit y por ultimo toarray
y_test= y_test.reshape(-1, 1)
y_test

array([['white'],
       ['white'],
       ['black'],
       ...,
       ['red'],
       ['white'],
       ['red']], dtype='<U5')

In [17]:
##3
enc = OneHotEncoder()
enc.fit(y_test)  

OneHotEncoder()

In [22]:
y_test = enc.transform(y_test).toarray()

In [23]:
#por fin y_test esta en la forma que queremos
y_test


array([[0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       ...,
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.]])

In [ ]:
#ahora hago lo mismo con y Y_train

In [24]:
y_train

array(['white', 'red', 'white', ..., 'black', 'black', 'white'],
      dtype='<U5')

In [25]:
y_train= y_train.reshape(-1, 1)

In [26]:
enc = OneHotEncoder()
enc.fit(y_train)  

OneHotEncoder()

In [27]:
y_train = enc.transform(y_train).toarray()

In [28]:
y_train

array([[0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       ...,
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.]])

In [ ]:
#ha funcionado. Tenemos y_test e y_train en el formato que necesitabamos
# podiamos haberlo hecho antes de dividir el split del train pero he decidido hacerlo dos veces para practicar

In [29]:
enc.get_feature_names()

array(['x0_black', 'x0_green', 'x0_red', 'x0_white'], dtype=object)

In [30]:
enc.inverse_transform([[0,1,0,0]])

array([['green']], dtype='<U5')

In [ ]:
#para correr el modelo

In [31]:
batch_size = 100
epochs = 20

model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, y_test))



Epoch 1/20
84/84 [==============================] - 13s 154ms/step - loss: 0.3390 - accuracy: 0.7171 - val_loss: 0.1954 - val_accuracy: 0.8493
Epoch 2/20
84/84 [==============================] - 14s 167ms/step - loss: 0.1843 - accuracy: 0.8686 - val_loss: 0.1106 - val_accuracy: 0.9185
Epoch 3/20
84/84 [==============================] - 14s 170ms/step - loss: 0.1274 - accuracy: 0.9118 - val_loss: 0.1026 - val_accuracy: 0.9247
Epoch 4/20
84/84 [==============================] - 15s 177ms/step - loss: 0.0948 - accuracy: 0.9374 - val_loss: 0.0565 - val_accuracy: 0.9623
Epoch 5/20
84/84 [==============================] - 14s 170ms/step - loss: 0.0784 - accuracy: 0.9500 - val_loss: 0.0467 - val_accuracy: 0.9704
Epoch 6/20
84/84 [==============================] - 15s 173ms/step - loss: 0.0695 - accuracy: 0.9557 - val_loss: 0.0375 - val_accuracy: 0.9790
Epoch 7/20
84/84 [==============================] - 15s 173ms/step - loss: 0.0594 - accuracy: 0.9616 - val_loss: 0.0409 - val_accuracy: 0.9776

In [33]:
X_test[45].shape
model.predict(np.array([X_test[403]]))

array([[3.6540296e-06, 4.2794942e-05, 7.4690814e-07, 9.9993980e-01]],
      dtype=float32)